<a href="https://colab.research.google.com/github/chaeyoung97/google-CoLab/blob/master/DACON_%EC%9C%84%EC%84%B1%EA%B4%80%EC%B8%A1_%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%99%9C%EC%9A%A9_%EA%B0%95%EC%88%98%EB%9F%89_%EC%82%B0%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive')


In [0]:
!unzip -q "/gdrive/My Drive/train.zip" -d "/train"

In [0]:
!unzip -q "/gdrive/My Drive/test.zip" -d "/test"

In [0]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import random
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Conv2DTranspose, MaxPooling2D, BatchNormalization, Activation, concatenate,Flatten,Dense, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.models import load_model, Model
from sklearn.model_selection import KFold
import warnings
 
warnings.filterwarnings("ignore")

In [0]:
# 재생산성을 위해 시드 고정
np.random.seed(7)
random.seed(7)
tf.random.set_seed(7)

#데이터 불러오기
- 0~10 채널만 불러오기

- 50개 이상의 픽셀에 강수량이 기록된 이미지만 사용

In [0]:
x_train =[]
y_train = []
X_test = []

def trainGenerator():
    train_path = '/train'
    train_files = sorted(glob.glob(train_path+ '/*'))

    #dqdm : for문의 상태바를 나타내는 라이브러리 
    for file in tqdm(train_files, desc = 'train'):
        dataset = np.load(file).astype('float32')
        target = dataset[:,:,-1].reshape(40,40,1)
        cutoff_labels = np.where(target<0, 0 ,target)
        feature = dataset[:,:,:10]
        if (cutoff_labels > 0).sum() >= 50:
            x_train.append(feature)
            y_train.append(target)



def testGenerator():
    test_path = '/test'
    test_files = sorted(glob.glob(test_path + '/*'))

    for file in tqdm(test_files, desc = 'test'):
        dataset = np.load(file).astype('float32')
        X_test.append(dataset[:,:,:10])
  

In [0]:
trainGenerator()
testGenerator()

In [0]:
#train = np.array(train)
test = np.array(X_test)
x_train = np.array(x_train)
y_train = np.array(y_train)

In [0]:
print("x_train: ",x_train.shape)
print("y_train: ",y_train.shape)
print("test: ",test.shape)

In [0]:
xtrain = x_train.copy()
ytrain = y_train.copy()

#Train data를 Train set 과 Test set으로 분리## 
- 여기선 test 셋이 따로 있어서 이 함수를 이 용도로 사용하지 않음
- 여기선 data를 늘릴 때 램 부족이 나지 않도록 하기 위해서 적절히 갯수를 나누기 위해 사용함


In [0]:

from sklearn.model_selection import train_test_split

def generate_data(x_train, y_train, t_r):
    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=t_r, random_state=64)
    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = generate_data(x_train, y_train,0.025)

print("x_train: ",x_train.shape)
print("y_train: ",y_train.shape)
print("x_test: ",x_test.shape)
print("y_test: ",y_test.shape)
print("test: ",test.shape)



#데이터 늘리기 
- train data를 회전시켜 data를 증가시킴
- data_rotate90 함수는 이미지를 시계반대 방향으로 90도 회전시킴
- data_rotate90(x_train , y_train, num)     함수의 세번째 인자인 num은 몇번 회전시킬건지를 나타냄

  ex) num=2 시계반대방향으로 180도 회전

- data_transpose 함수는 이미지를 반전시킴

In [0]:
def data_rotate90(x_train , y_train, num):
    rotate_x = np.zeros_like(x_train)
    rotate_y = np.zeros_like(y_train)

    for i in range(x_train.shape[0]):
        x = np.zeros([40, 40, 10])
        y = np.zeros([40, 40,1])
        for j in range(10):
            x[:,:,j]=np.rot90(x_train[i,:,:,j], num)
        y[:,:,0]=np.rot90(y_train[i,:,:,0], num)

        rotate_x[i,:,:,:] = x
        rotate_y[i,:,:,:] = y

    return rotate_x, rotate_y


def data_transpose(x_train, y_train):
    x = np.zeros_like(x_train)
    y = np.zeros_like(y_train)
    for i in range(x_train.shape[0]):
        for j in range(x_train.shape[3]):
            x[i,:,:,j] = x_train[i,:,:,j].T
        y[i,:,:,0] = y_train[i,:,:,0].T

    return x , y

#Loss Function 

In [0]:
from sklearn.metrics import f1_score

def mae(y_true, y_pred) :
    
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    
    y_pred = y_pred.reshape(1, -1)[0]
    
    over_threshold = y_true >= 0.1
    
    return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))

def fscore(y_true, y_pred):
    
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    
    y_pred = y_pred.reshape(1, -1)[0]
    
    remove_NAs = y_true >= 0
    
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    
    return(f1_score(y_true, y_pred))

def maeOverFscore(y_true, y_pred):
    
    return mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)

def fscore_keras(y_true, y_pred):
    score = tf.py_function(func=fscore, inp=[y_true, y_pred], Tout=tf.float32, name='fscore_keras')
    return score

def maeOverFscore_keras(y_true, y_pred):
    score = tf.py_function(func=maeOverFscore, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_mse') 
    return score

#평가 함수 

In [0]:
def mae_over_fscore(y_true, y_pred):
    '''
    y_true: sample_submission.csv 형태의 실제 값
    y_pred: sample_submission.csv 형태의 예측 값
    '''


    y_true = np.array(y_true)
    y_true = y_true.reshape(1, -1)[0]  
    
    y_pred = np.array(y_pred)
    y_pred = y_pred.reshape(1, -1)[0]
    
    # 실제값이 0.1 이상인 픽셀의 위치 확인
    IsGreaterThanEqualTo_PointOne = y_true >= 0.1
    
    # 실제 값에 결측값이 없는 픽셀의 위치 확인 
    IsNotMissing = y_true >= 0
    
    # mae 계산
    mae = np.mean(np.abs(y_true[IsGreaterThanEqualTo_PointOne] - y_pred[IsGreaterThanEqualTo_PointOne]))
    
    # f1_score 계산 위해, 실제값에 결측값이 없는 픽셀에 대해 1과 0으로 값 변환
    y_true = np.where(y_true[IsNotMissing] >= 0.1, 1, 0)
    
    y_pred = np.where(y_pred[IsNotMissing] >= 0.1, 1, 0)
    
    # f1_score 계산    
    f_score = f1_score(y_true, y_pred) 
    
    # f1_score가 0일 나올 경우를 대비하여 소량의 값 (1e-07) 추가 
    return mae / (f_score + 1e-07) 

#모델 만들기

In [0]:
def build_model(x):
   
    input_layer = Input((40, 40, 10))
    ########################################
    batch = BatchNormalization()(input_layer)
    conv1 = Conv2D(x, (1,1), strides= 1, padding = 'same', activation = 'relu')(batch)
    batch = BatchNormalization()(conv1)
    conv2 = Conv2D(x/2, (2,2), strides= 1, padding = 'same', activation = 'relu' )(batch)
    concat = concatenate([conv1, conv2], axis= 3)
    ########################################
    batch = BatchNormalization()(concat)
    conv3 = Conv2D(x/4, (3,3), strides= 1, padding = 'same', activation = 'relu')(batch)
    concat = concatenate([concat, conv3])
    ########################################
    batch = BatchNormalization()(concat)
    conv4 = Conv2D(x/8, (3,3), strides= 1, padding = 'same', activation = 'relu' )(batch)
    concat = concatenate([concat, conv4], axis= 3)
    ########################################
    batch = BatchNormalization()(concat)
    conv5 = Conv2D(32, (3,3), strides= 1, padding = 'same', activation = 'relu' )(batch)
    concat = concatenate([concat, conv5], axis= 3)
    batch = BatchNormalization()(concat)
    conv55 = Conv2D(x/8, (3,3), strides= 1, padding = 'same', activation = 'relu' )(batch)
    concat = concatenate([concat, conv55], axis= 3)
   ########################################
    batch = BatchNormalization()(concat)
    conv6 = Conv2D(32, (3,3), strides= 1, padding = 'same', activation = 'relu' )(batch)
    concat = concatenate([concat, conv6], axis= 3)
    ########################################
    batch = BatchNormalization()(concat)
    
    output_layer = Conv2D(1, (1,1), strides= 1, padding= 'same', activation= 'relu')(batch)
   
    model = Model(input_layer,  output_layer)

    return model

# 데이터 늘리기 함수를 이용해 새로운data를 만들고  기존의 train data와 합치기

- data 늘리기

In [0]:
## 반시계 방향으로 90도 회전한 data
x_90, y_90 = data_rotate90(x_train, y_train, 1)

In [0]:
## 반시계 방향으로 180도 회전한 data
x_180, y_180 = data_rotate90(x_90, y_90, 1)

In [0]:
##반시계 방향으로 270도 회전한 data
x_270, y_270 = data_rotate90(x_180, y_180 ,1)

In [0]:
##270도 회전한 data 뒤집기
x_t270, y_t270 = data_transpose(x_270, y_270)

##90도 회전한 data 뒤집기 (램 부족으로 추가 못함)
#x_t90, y_t90 = data_transpose(x_90, y_90)

##180도 회전한 data 뒤집기 (램 부족으로 추가 못함)
#x_t180, y_t180 = data_transpose(x_180, y_180)

- data 합치기

In [0]:
x_train = np.concatenate((x_train, x_90, x_180, x_270, x_t270))
del x_90, x_180, x_270, x_t270

In [0]:
y_train = np.concatenate((y_train, y_90, y_180, y_270, y_t270))
del y_90, y_180, y_270, y_t270

In [0]:
print('x_train: ', x_train.shape)
print('y_train: ', y_train.shape)

#Ensemble 앙상블로 정확도 올리기

In [0]:
models = []
num_models = 3
x=1024
t =2;
for i in range(num_models):
    models.append(build_model(x/t))
    t = t*2

In [0]:
#models[1].summary()

- 모델 별로 학습시키고 학습된 모델 저장하기

In [0]:
m = 0
for model in models:
    model.compile(loss="mae", optimizer="adam", metrics=[maeOverFscore_keras, fscore_keras])
    model.fit(x_train, y_train, epochs= 30, batch_size = 64)

    

- 학습된 모델 불러와서 모델 평가하기
- 이때 세 모델의 값의 평균을 사용함

In [0]:
pred = []
m = 0
for model in models:
    pred.append(model.predict(x_test))
    print(mae_over_fscore(y_test, pred[m]))
    m = m + 1
prediction= models[0].predict(test)


#Submission

In [0]:
submission = pd.read_csv('/content/drive/My Drive/sample_submission.csv')

In [0]:
submission.iloc[:,1:] = prediction.reshape(-1, 1600)

In [0]:
submission.to_csv('/content/drive/My Drive/Dacon_submission3.csv', index= False)